In [1]:
import numpy as np

In [3]:
# will return set of images of cardinality = num, each image has channels=3, width=3, height = 4
def get_set(num, w, h):
    set_ = []
    for i in xrange(num):
        image = []
        for j in xrange(3):
            channel = []
            for k in xrange(w):
                col = []
                for l in xrange(h):
                    col.append((i+1) * 1000 + (j+1) * 100 + (k+1) * 10 + (l+1))
                channel.append(col)
            channel = np.array(channel).transpose()
            image.append(channel)
        set_.append(image)
    set_=np.array(set_)
    return set_
set_ = get_set(8, 6, 6)
set_[0]

array([[[1111, 1121, 1131, 1141, 1151, 1161],
        [1112, 1122, 1132, 1142, 1152, 1162],
        [1113, 1123, 1133, 1143, 1153, 1163],
        [1114, 1124, 1134, 1144, 1154, 1164],
        [1115, 1125, 1135, 1145, 1155, 1165],
        [1116, 1126, 1136, 1146, 1156, 1166]],

       [[1211, 1221, 1231, 1241, 1251, 1261],
        [1212, 1222, 1232, 1242, 1252, 1262],
        [1213, 1223, 1233, 1243, 1253, 1263],
        [1214, 1224, 1234, 1244, 1254, 1264],
        [1215, 1225, 1235, 1245, 1255, 1265],
        [1216, 1226, 1236, 1246, 1256, 1266]],

       [[1311, 1321, 1331, 1341, 1351, 1361],
        [1312, 1322, 1332, 1342, 1352, 1362],
        [1313, 1323, 1333, 1343, 1353, 1363],
        [1314, 1324, 1334, 1344, 1354, 1364],
        [1315, 1325, 1335, 1345, 1355, 1365],
        [1316, 1326, 1336, 1346, 1356, 1366]]])

In [4]:
mask_same = np.array([[000,000,000,000,111,112,000,121,122,000,000,000,000,211,212,000,221,222,000,000,000,000,311,312,000,321,322],
                      [000,000,000,112,113,114,122,123,124,000,000,000,212,213,214,222,223,224,000,000,000,312,313,314,322,323,324],
                      [000,000,000,114,115,116,124,125,126,000,000,000,214,215,216,224,225,226,000,000,000,314,315,316,324,325,326],
                      [000,121,122,000,131,132,000,141,142,000,221,222,000,231,232,000,241,242,000,321,322,000,331,332,000,341,342],
                      [122,123,124,132,133,134,142,143,144,222,223,224,232,233,234,242,243,244,322,323,324,332,333,334,342,343,344],
                      [124,125,126,134,135,136,144,145,146,224,225,226,234,235,236,244,245,246,324,325,326,334,335,336,344,345,346],
                      [000,141,142,000,151,152,000,161,162,000,241,242,000,251,252,000,261,262,000,341,342,000,351,352,000,361,362],
                      [142,143,144,152,153,154,162,163,164,242,243,244,252,253,254,262,263,264,342,343,344,352,353,354,362,363,364],
                      [144,145,146,154,155,156,164,165,166,244,245,246,254,255,256,264,265,266,344,345,346,354,355,356,364,365,366]])

In [5]:
mask_valid = np.array([[111,112,113,121,122,123,131,132,133,211,212,213,221,222,223,231,232,233,311,312,313,321,322,323,331,332,333],
                       [113,114,115,123,124,125,133,134,135,213,214,215,223,224,225,233,234,235,313,314,315,323,324,325,333,334,335],
                       [131,132,133,141,142,143,151,152,153,231,232,233,241,242,243,251,252,253,331,332,333,341,342,343,351,352,353],
                       [133,134,135,143,144,145,153,154,155,233,234,235,243,244,245,253,254,255,333,334,335,343,344,345,353,354,355]])

In [6]:
# block will create input matrix of size (3*6*6, num) where 3*6*6 is size of each image and num is number of images 

def convert_channel_to_column(channel):
    return np.ravel(channel, order='F')

def convert_image_to_column(image):
    result = []
    for channel in image:
        result += convert_channel_to_column(channel).tolist()
    return result

def convert_set_to_matrix(set_):
    result = []
    for image in set_:
        result += convert_image_to_column(image)
    return np.reshape(np.array(result), [3*6*6, set_.shape[0]], order='F')
                       
input_matrix = convert_set_to_matrix(set_)

In [7]:
# block will create output matrix of size (3*(6/2)*(6/2)*9, num) where 3 is number of channels in input, 
# 6 is width/height of input 6/2 is width/height of output and 9 is kenrelWidth * kernelHeight

def create_output_per_image(index, mask):
    result = []
    for elem in mask:
        if elem != 0:
            elem = elem + 1000*index
        result.append(elem)
    return result

def create_output_of_set(set_, mask):
    result = []
    for idx, image in enumerate(set_):
        result += create_output_per_image(idx+1, mask)
    return np.reshape(np.array(result), [3*(6/2)*(6/2)*9, set_.shape[0]], order='F')

output_matrix_same = create_output_of_set(set_, np.ravel(mask_same).tolist())

In [8]:
# block will create output matrix of size (3*((6-3+1)/2)*((6-3+1)/2)*9, num) where 3 is number of channels in input, 
# 6 is width/height of input (6-3+1)/2 is width/height of output and 9 is kenrelWidth * kernelHeight

def create_output_per_image(index, mask):
    result = []
    for elem in mask:
        if elem != 0:
            elem = elem + 1000*index
        result.append(elem)
    return result

def create_output_of_set(set_, mask):
    result = []
    for idx, image in enumerate(set_):
        result += create_output_per_image(idx+1, mask)
    return np.reshape(np.array(result), [3*((6-3+1)/2)*((6-3+1)/2)*9, set_.shape[0]], order='F')

output_matrix_valid = create_output_of_set(set_, np.ravel(mask_valid).tolist())

In [9]:
np.savetxt('/home/ahsan/squirrel_latest/squirrel/trunk/src/Test/test-case-data/prepare-conv-strided-forw-i.txt', input_matrix, fmt='%i', delimiter=' ', header='108 8')
np.savetxt('/home/ahsan/squirrel_latest/squirrel/trunk/src/Test/test-case-data/prepare-conv-strided-same-forw-r.txt', output_matrix_same, fmt='%i', delimiter=' ', header='243 8')
np.savetxt('/home/ahsan/squirrel_latest/squirrel/trunk/src/Test/test-case-data/prepare-conv-strided-valid-forw-r.txt', output_matrix_valid, fmt='%i', delimiter=' ', header='108 8')